In [1]:
%load_ext autoreload

In [8]:
import os
import pandas as pd
from common import sessions, sessfile, load_metadata
root = '/Users/lukearend/phd/kfx/data/mnt/Ketamine'

In [3]:
from tqdm import tqdm

metadata = {}
missing = '/Users/lukearend/phd/kfx/ref/2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.meta'
for region in ['hpc', 'pfc']:
    metadata[region] = {}
    for signal in ['lfp', 'ap']:
        df = []
        for i, sess in tqdm(sessions.iterrows(), total=36):
            metafile = sessfile(sess.path, region, signal, 'meta')
            path = os.path.join(root, metafile)
            try:
                md = pd.Series(load_metadata(path))
            except FileNotFoundError:
                md = pd.Series(load_metadata(missing))
            df.append(md)
        metadata[region][signal] = pd.DataFrame(df, index=sessions.index)

100%|███████████████████████████████████████████| 36/36 [00:03<00:00, 10.81it/s]


In [24]:
apsr = 30000
lfsr = 2500
apgain = 500
lfgain = 125
int2volt = 0.6 / 512

In [51]:
dfs = []
for signal in ['lfp', 'ap']:
    for region in ['hpc', 'pfc']:
        df = metadata[region][signal][['fileCreateTime', 'fileSizeBytes', 'fileTimeSecs', 'firstSample']]
        df.insert(0, 'region', region)
        df.insert(0, 'signal', signal)
        dfs.append(df.reset_index())
df = pd.concat(dfs)
df = df.astype({'fileSizeBytes': 'int', 'fileTimeSecs': 'float', 'firstSample': 'int'})
df

,recording,signal,region,fileCreateTime,fileSizeBytes,fileTimeSecs,firstSample
0,0,lfp,hpc,2022-07-27T19:41:37,13967783830,7255.991600,695169
1,1,lfp,hpc,2022-07-28T13:19:08,14372062320,7466.006400,167143
2,2,lfp,hpc,2022-08-01T16:30:01,14005579280,7275.625600,280726
3,3,lfp,hpc,2022-08-03T12:45:02,13967481990,7255.834800,96437
4,4,lfp,hpc,2022-08-04T15:45:22,13976037460,7260.279200,107763
...,...,...,...,...,...,...,...
31,31,ap,pfc,2022-10-21T13:15:23,209196445920,9056.123200,5413200
32,32,ap,pfc,2022-10-24T14:35:09,167409641630,7247.170633,6331397
33,33,ap,pfc,2022-10-25T16:00:02,167600188140,7255.419400,5234238
34,34,ap,pfc,2022-10-27T15:45:10,209420080100,9065.804333,13847174


In [68]:
df.fileSizeBytes.sum() / 1e9

14878.07950013

all matrices have 385 columns
lfp have 20 million rows, 14 GB, 150k rows / min
ap have 220 million rows, 170 GB, 1.8m rows / min

recordings are taken in PFC and HPC
LFP and AP are recorded simultaneously
session has three or four epochs
each epoch is 30 mins
each epoch is 30 mins
1 min:
    PFC:
        AP: 1.8m rows, 385 cols
        LFP: 150k rows, 385 cols
    HPC:
        AP: 1.8m rows, 385 cols
        LFP: 150k rows, 385 cols
1.95m rows per min
each row is 385 * 2 bytes
1.5015 GB / min
45.045 GB / epoch